In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


In [4]:
df = pd.read_csv(csv_file)

In [11]:
# one-hot encoding for categorical features
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [12]:
target = df.pop('target')

## Load data using tf.data.Dataset

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [14]:
train_dataset = dataset.shuffle(len(df)).batch(32)

In [15]:
# Create model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [16]:
model.fit(train_dataset, epochs=15, verbose=0)

## Alternative to feature columns

Passing a dictionary as an input to a model is as easy as creating a matching dictionary of `tf.keras.layers.Input` layers, applying any pre-processing and stacking them up using the `functional api`. You can use this as an alternative to feature columns.

In [18]:
# First define the inputs
inputs = {key: tf.keras.Input(shape=(), name=key) for key in df.keys()}

In [19]:
inputs

{'age': <tf.Tensor 'age:0' shape=(None,) dtype=float32>,
 'sex': <tf.Tensor 'sex:0' shape=(None,) dtype=float32>,
 'cp': <tf.Tensor 'cp:0' shape=(None,) dtype=float32>,
 'trestbps': <tf.Tensor 'trestbps:0' shape=(None,) dtype=float32>,
 'chol': <tf.Tensor 'chol:0' shape=(None,) dtype=float32>,
 'fbs': <tf.Tensor 'fbs:0' shape=(None,) dtype=float32>,
 'restecg': <tf.Tensor 'restecg:0' shape=(None,) dtype=float32>,
 'thalach': <tf.Tensor 'thalach:0' shape=(None,) dtype=float32>,
 'exang': <tf.Tensor 'exang:0' shape=(None,) dtype=float32>,
 'oldpeak': <tf.Tensor 'oldpeak:0' shape=(None,) dtype=float32>,
 'slope': <tf.Tensor 'slope:0' shape=(None,) dtype=float32>,
 'ca': <tf.Tensor 'ca:0' shape=(None,) dtype=float32>,
 'thal': <tf.Tensor 'thal:0' shape=(None,) dtype=float32>}

In [24]:
# >>> tf.stack(list(inputs.values()), axis=-1)
# <tf.Tensor 'stack:0' shape=(None, 13) dtype=float32>

In [29]:
# Concatenate all inputs
x = tf.stack(list(inputs.values()), axis=-1)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs=inputs, outputs=output)

In [34]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age (InputLayer)                [(None,)]            0                                            
__________________________________________________________________________________________________
sex (InputLayer)                [(None,)]            0                                            
__________________________________________________________________________________________________
cp (InputLayer)                 [(None,)]            0                                            
__________________________________________________________________________________________________
trestbps (InputLayer)           [(None,)]            0                                            
______________________________________________________________________________________________

In [31]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict(orient='list'), target.values)).batch(16)

In [35]:
model.fit(dict_slices, epochs=15)

Train for 19 steps
Epoch 1/15
19/19 [==============================] - 0s 20ms/step - loss: 7.5308 - accuracy: 0.6271
Epoch 2/15
19/19 [==============================] - 0s 4ms/step - loss: 7.0010 - accuracy: 0.5479
Epoch 3/15
19/19 [==============================] - 0s 4ms/step - loss: 6.8544 - accuracy: 0.5479
Epoch 4/15
19/19 [==============================] - 0s 4ms/step - loss: 6.7104 - accuracy: 0.5512
Epoch 5/15
19/19 [==============================] - 0s 4ms/step - loss: 6.5708 - accuracy: 0.5545
Epoch 6/15
19/19 [==============================] - 0s 4ms/step - loss: 6.4314 - accuracy: 0.5545
Epoch 7/15
19/19 [==============================] - 0s 4ms/step - loss: 6.2925 - accuracy: 0.5611
Epoch 8/15
19/19 [==============================] - 0s 4ms/step - loss: 6.1543 - accuracy: 0.5611
Epoch 9/15
19/19 [==============================] - 0s 4ms/step - loss: 6.0168 - accuracy: 0.5611
Epoch 10/15
19/19 [==============================] - 0s 4ms/step - loss: 5.8799 - accuracy: 0.5611